In [893]:
import numpy as np
import matplotlib.pyplot as plt
import numpy.polynomial.legendre as leg

Set up classes

In [894]:
class particle:
     def __init__(self, alive, mu, x, g, w):
        self.alive = alive
        self.mu = mu
        self.x = x
        self.g = g
        self.w = w

In [895]:
class source:
    def __init__(self, scatter, fission):
        self.scatter = scatter
        self.fission = fission

In [896]:
class tally:
    def __init__(self, scalar_flux, current, k):
        self.scalar_flux = scalar_flux
        self.current = current
        self.k = k
    
    def zero_g(self, g):
        self.scalar_flux[:, g] *= 0
        self.current[:, g] *= 0

    def zero(self):
        self.scalar_flux *= 0
        self.current *= 0

In [897]:
class material:
    def __init__(self, nuSigmaF, SigmaS, SigmaT):
        self.nuSigmaF = nuSigmaF
        self.SigmaS = SigmaS
        self.SigmaT = SigmaT

In [898]:
class cell:
    def __init__(self, left, right, material):
        self.left = left
        self.right = right
        self.material = material

In [899]:
class mesh:
    def __init__(self, Nx, L, bc, U, W):
        self.Nx = Nx
        self.L = L
        self.dx = L/(Nx-1)
        self.mesh_x = np.linspace(0, L, Nx)
        self.mesh_flux = np.linspace(0, L, Nx-1)
        self.bc = bc
        self.U = U
        self.W = W

In [900]:
class bc:
    def __init__(self, bc):
        self.bc = bc

Set up functions

In [901]:
def zero_tally(tally):
    tally.scalar_flux = np.zeros_like(tally.scalar_flux)
    tally.current = np.zeros_like(tally.current)

In [902]:
def f_convergence(new, old):
    # convergence criteria
    epi = 1e-5
    if np.amax(abs(new - old)) < epi:
        return True
    else:
        return False

In [903]:
def k_convergence(new, old):
    # convergence criteria
    ek = 1e-5
    if np.amax(abs((new - old) / new)) < ek:
        return True
    else:
        return False

In [904]:
def si_convergence(new, old):
    # convergence criteria
    esi = 1e-6
    if np.amax(abs(new - old) / new) < esi:
        return True
    else:
        return False

In [905]:
def get_surface(p, bin_id):
    if p.mu > 0:
        return bin_id+1
    else:
        return bin_id

In [906]:
def get_material(p, cells):
    for i in range(len(cells)):
        right = cells[i].right
        left = cells[i].left
        if p.x >= left and p.x < right:
            return cells[i].material

In [907]:
def calculate_fission_source():
    # get values
    flux = tally1.scalar_flux
    source = np.zeros(mesh1.Nx-1)
    dx = mesh1.dx
    index = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        nuSigmaF = material.nuSigmaF
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            source[j] = nuSigmaF[0]*flux[j,0] + nuSigmaF[1]*flux[j,1]
            
        index = j+1
        
    return source

In [908]:
def calculate_k(fission_new, fission_old):
    dx = mesh1.dx
    k_new = tally1.k * np.sum(dx*fission_new)/np.sum(dx*fission_old)
    return k_new

In [909]:
def calculate_source_S(g):
    # only has fission source in fast group
    if g == 0: 
        Sg = 1/tally1.k * source1.fission
    else:
        Sg = np.zeros_like(source1.fission)

    # get other values
    if g == 0:
        g_prime = 1
    else:
        g_prime = 0
    scalar_flux = tally1.scalar_flux[:,g_prime]
    dx = mesh1.dx
    index = 0
    
    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Sg[j] += SigmaS[g_prime, g] * scalar_flux[j]
        index = j+1

    return Sg

In [910]:
def calculate_source_Q(Sg, g):
    # get values
    Qg = Sg.copy()
    scalar_flux = tally1.scalar_flux[:,g]
    dx = mesh1.dx
    index = 0

    # loop through all cells
    for i in range(len(cells)):
        # get cell and material data
        left = cells[i].left
        right = cells[i].right
        material = cells[i].material
        SigmaS = material.SigmaS
        bins = int((right - left) / dx)
        
        # calculate source for current cell
        for j in range(bins):
            j += index
            Qg[j] += SigmaS[g, g] * scalar_flux[j]
        index = j+1
        

    return Qg

In [911]:
def outgoing_flux(Q, SigmaT, dx, u, angular_flux_inc):
    tau = SigmaT * dx / abs(u)
    edge_flux = angular_flux_inc * np.exp(-tau) + Q/SigmaT * (1 - np.exp(-tau))
    return edge_flux

In [912]:
def cell_average_flux(Q, SigmaT, dx, u, angular_flux_inc, angular_flux_exit):
    cell_flux = Q/SigmaT - (u * (angular_flux_exit - angular_flux_inc) / (dx * SigmaT))
    return cell_flux

In [913]:
def sweep(bc_incoming, Q, n, g):
    u = mesh1.U[n]
    w = mesh1.W[n]
    angular_flux_inc = bc_incoming

    # sweep right
    if u > 0:
        index = 0
        tally1.current[0, g] += w * u * angular_flux_inc
        # loop through all cells
        for i in range(len(cells)):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)

            # calculate source for current cell
            for j in range(bins):
                j += index
                # compute cell edge and cell average fluxes, save outgoing flux for next cell
                angular_flux_exit = outgoing_flux(Q[j], SigmaT, dx, u, angular_flux_inc)
                cell_average = cell_average_flux(Q[j], SigmaT, dx, u, angular_flux_inc, angular_flux_exit)
                angular_flux_inc = angular_flux_exit

                # tally flux and current
                tally1.scalar_flux[j, g] += w * cell_average
                tally1.current[j + 1, g] += w * u * angular_flux_exit

            index = j+1
    
    # sweep left
    if u < 0:
        index = mesh1.Nx - 2
        tally1.current[index, g] += w * u * angular_flux_inc
        # loop through all cells
        for i in range(len(cells)-1,-1,-1):
            # get cell and material data
            dx = mesh1.dx
            left = cells[i].left
            right = cells[i].right
            material = cells[i].material
            SigmaT = material.SigmaT[g]
            bins = int((right - left) / dx)

            # calculate source for current cell
            for j in range(bins):
                j = index - j

                # compute cell edge and cell average fluxes, save outgoing flux for next cell
                angular_flux_exit = outgoing_flux(Q[j], SigmaT, dx, u, angular_flux_inc)
                cell_average = cell_average_flux(Q[j], SigmaT, dx, u, angular_flux_inc, angular_flux_exit)
                angular_flux_inc = angular_flux_exit 

                # tally flux and current
                tally1.scalar_flux[j, g] += w * cell_average
                tally1.current[j - 1, g] += w * u * angular_flux_exit

            index -= j-1

    return angular_flux_exit

Main loops

In [914]:
def source_iteration():
    iter = 100
    scalar_flux_old = np.zeros_like(mesh1.mesh_flux)
    G = np.size(tally1.scalar_flux, axis=1)

    for g in range(G):
        Sg = calculate_source_S(g)
        #print("SG: ", Sg)

        for i in range(iter):
            Qg = calculate_source_Q(Sg, g)
            #print("Q: ", Qg)
            tally1.zero_g(g)

            # sweep all angles
            for n in range(len(mesh1.U)):
                bc_in = np.flip(range(len(mesh1.U)))[n]
                bc1.bc[g, n] = sweep(bc1.bc[g, bc_in], Qg, n, g)
            
            # check convergence
            if si_convergence(tally1.scalar_flux[:,g], scalar_flux_old):
                break

            scalar_flux_old = tally1.scalar_flux[:,g].copy()
            #print("flux: ", tally1.scalar_flux)

In [915]:
def power_iteration():
    iter = 100
    k_old = 1
    f_old = np.ones_like(source1.fission)
    for i in range(iter):
        source_iteration()

        source1.fission = calculate_fission_source()
        tally1.k = calculate_k(source1.fission, f_old)

        if f_convergence(source1.fission, f_old) and k_convergence(tally1.k, k_old):
            break
        
        f_old = source1.fission.copy()
        k_old = tally1.k.copy()
        
        print("power", i)


In [920]:
def main_loop():
    power_iteration()
    print(tally1.scalar_flux)
    print(tally1.current)

Test

In [921]:
# materials
mox = material(np.array([0.0,0.0]), 
              np.array([[0.2,0.2],[0.0,0.2]]), 
              np.array([1.0,1.0]))
# cells
cells = [cell(0, 10, mox)]

# gauss-legendre
U, W = leg.leggauss(2)

# mesh
mesh1 = mesh(20 + 1, 10, "reflective", U, W)

# initialize tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
            np.zeros((mesh1.Nx, 2)),
            1)

# initialize source
source1 = source(np.zeros_like(mesh1.mesh_flux), 
                np.ones_like(mesh1.mesh_flux))

# initialize bc
bc1 = bc(np.zeros((2, len(mesh1.U))))

In [922]:
main_loop()

[[3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333276 2.22222146]
 [3.33333275 2.22222145]
 [3.33333275 2.22222144]
 [3.33333275 2.22222144]
 [3.33333278 2.22222145]
 [3.33333283 2.22222153]
 [3.33333288 2.22222165]
 [3.3333328  2.22222163]
 [3.33333244 2.22222114]
 [3.33333226 2.22222062]
 [3.33333352 2.22222238]
 [3.33333223 2.2222207 ]
 [3.33333336 2.2222221 ]]
[[-3.22331051e-12 -7.96362976e-12]
 [-1.98785433e-12 -2.02068362e-11]
 [ 2.22057928e-11 -1.55884194e-11]
 [ 8.92101948e-11  3.69316799e-11]
 [ 2.19527396e-10  1.92556748e-10]
 [ 3.97349265e-10  5.07782483e-10]
 [ 4.83049600e-10  9.38821354e-10]
 [ 7.84399212e-11  1.10086762e-09]
 [-1.53519575e-09 -7.08150205e-11]
 [-4.99879771e-09 -4.31576319e-09]
 [-9.33610211e-09 -1.24529491e-08]
 [-9.06397124e-09 -1.95411294e-08]
 [ 7.08873582e-09 -8.51368587e-09]
 [ 4.57991728e-08  4.31179762e-08]
 [ 7.5892

Problem A

In [919]:
# materials
mox = material(np.array([0,1.05]), 
              np.array([[0.25,0],[0,0]]), 
              np.array([0.25,0.75]))

u = material(np.array([0,0.40]), 
              np.array([[0.25,0],[0,0]]), 
              np.array([0.25,0.25]))

water = material(np.array([0,0]), 
              np.array([[0.20,0.05],[0,1.25]]), 
              np.array([0.25,1.25]))
# cells
cells = []

for i in range(8):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, mox), cell(i+.9375, i+1.25, water)])

for i in range(8, 16):
    i *= 1.25
    cells.extend([cell(i+0, i+.3125, water), cell(i+.3125, i+.9375, u), cell(i+.9375, i+1.25, water)])

# gauss-legendre
U, W = leg.leggauss(10)

# mesh
mesh1 = mesh(128 + 1, 20, "reflective", U, W)

# tally
tally1 = tally(np.zeros((mesh1.Nx-1, 2)), 
               np.zeros((mesh1.Nx, 2)),
               1)